In [ ]:
#import data visualisation and handlng libraries 
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from openpyxl import load_workbook
from openpyxl import Workbook
import openpyxl
import os
import re
from fancyimpute import IterativeImputer
from sklearn.preprocessing import LabelEncoder
import nltk
from nltk.tokenize import word_tokenize
import numpy as np

nltk.download('punkt')

(1) Problem 1

In [ ]:
#load od1.xlsx and create a worksheet
wb1 = load_workbook(filename = "od1.xlsx")
ws1 = wb1["Sheet1"]

#load od2.xlsx and create a worksheet
wb2 = load_workbook(filename = "od2.xlsx")
ws2 = wb2["Sheet1"]

#create a new third workbook and sheet which will  be used to store the merged data
merged_wb = Workbook()
merged_ws = merged_wb.active

#check if od.xlsx exists and delete
if os.path.isfile("od.xlsx"):

    print("deleted exisitng file")
    os.remove("od.xlsx")

#loop for rows in a worksheet using iter_rows
#start at row 2 as row 1 is column headings
#values_only = True used to compare cell values rather than cell objects
for row1 in ws1.iter_rows(min_row = 2, values_only = True):

    for row2 in ws2.iter_rows(min_row = 2, values_only = True):
        
        #check if shared columns are equal for a row
        if row1[3] == row2[0] and row1[4] == row2[1] and row1[5] == row2[2] and row1[6] == row2[3] and row1[7] == row2[7]:

            #append row to worksheet if equal
            merged_ws.append(row1[0:7] + row2[4:])

row_num = 0

#regular expression to match ID
id_regex = r'#(\d+)'

#dictionary to store ids
id_dict = {}

#loop for rows in merged sheet starting at 1 as no column heading exist yet
for row in merged_ws.iter_rows(min_row = 1):
    
    #re. search returns a match object
    #.group returns the string if there is a match
    id_val = re.search(id_regex, row[0].value).group(1)

    #convert string to int
    id_dict[row_num] = int(id_val)

    row_num += 1

#create a list containing the order the rows should be in
sorted_rows = [r[0] for r in sorted(id_dict.items(), key = lambda x: x[1])]

for idx, row_num in enumerate(sorted_rows):

    #move the rows so they are sorted after row 244
    merged_ws.move_range(f"A{row_num + 1}:M{row_num + 1}", rows = 245 - row_num + idx)

#remove the now empty rows 1-244
merged_ws.delete_rows(1,245)

#insert a new row for the headers
merged_ws.insert_rows(1)
header_row = merged_ws[1]

#header titles
headers = ["ID", "Age", "Gender", "Diagnosis", "dioptre_1", "dioptre_2", "astigmatism", "Phakic/Pseudophakic", "Pneumatic", "Perkins", "Pachymetry", "Axial_Length", "VF_MD"]

#add the headers to the header row
for header in headers:

    header_row[headers.index(header)].value = header

#loop through the cells in the header row
for cell in header_row:

    #format header row
    cell.font = openpyxl.styles.Font(bold = True)
    cell.alignment = openpyxl.styles.Alignment(horizontal = 'center')
    border = openpyxl.styles.Border(left=openpyxl.styles.Side(style='thin'),right=openpyxl.styles.Side(style='thin'),top=openpyxl.styles.Side(style='thin'),bottom=openpyxl.styles.Side(style='thin'))
    cell.border = border

#insert row for grouped headers
merged_ws.insert_rows(1)
grouped_row = merged_ws[1]

#grouped header titles
grouped_headers = ["Age", "Gender", "Diagnosis", "Refractive_Defect", "", "", "Phakic/Pseudophakic", "IOP", "", "Pachymetry", "Axial_Length", "VF_MD"]

#add the headers to the grouped headers row
for header in grouped_headers:

    grouped_row[grouped_headers.index(header) + 1].value = header

#loop through each cell in the grouped header row
for cell in grouped_row:

    #format the grouped headers
    cell.font = openpyxl.styles.Font(bold = True)
    cell.alignment = openpyxl.styles.Alignment(horizontal = 'center')
    border = openpyxl.styles.Border(left=openpyxl.styles.Side(style='thin'),right=openpyxl.styles.Side(style='thin'),top=openpyxl.styles.Side(style='thin'),bottom=openpyxl.styles.Side(style='thin'))
    cell.border = border

#load os sheet
oswb = load_workbook(filename = "os.xlsx")
osws = oswb["Sheet1"]

#merge cells to created grouped headers
merged_ws.merge_cells('E1:G1')
merged_ws.merge_cells('I1:J1')

#loop through the columns in os.xslx
for column in osws.columns:

    #get the width of each column
    col_index = column[1].column_letter
    col_width = osws.column_dimensions[col_index].width

    #set column width for od equal to os columns
    if col_width == 13.0:

        merged_ws.column_dimensions[col_index].width = col_width /1.475

    else:
        #set column width for od equal to os columns
        merged_ws.column_dimensions[col_index].width = col_width

#save sheet to workbook
merged_wb.save('od.xlsx')

(2) Problem 2

In [ ]:
od_data = pd.read_excel("od.xlsx")
os_data = pd.read_excel("os.xlsx")

od_data['Axial_Length'].fillna(26, inplace = True)
os_data['Axial_Length'].fillna(26, inplace = True)

od_id = od_data.iloc[1:, 0].reset_index(drop = True)
os_id = os_data.iloc[1:, 0].reset_index(drop = True)

od_data.drop('Unnamed: 0', axis = 1, inplace = True)
os_data.drop('Unnamed: 0', axis = 1, inplace = True)

od_data.drop(index=0, inplace=True)
os_data.drop(index=0, inplace=True)

od_data.reset_index()
os_data.reset_index()

#print(os_data['Diagnosis'].value_counts(dropna = False))
#print(od_data['Phakic/Pseudophakic'].value_counts(dropna = False))
#print(os_data['Phakic/Pseudophakic'].value_counts(dropna = False))

diagnosis_mapping = {
    'Healthy': 'healthy',
    'heal.': 'healthy',
    'heaLthy': 'healthy',
    'healthY': 'healthy',
    'sus': 'suspicious',
    'Suspicious': 'suspicious',
    'Glaucoma': 'glaucoma',
    'glau.': 'glaucoma'
}

os_data['Diagnosis'] = os_data['Diagnosis'].apply(lambda x: ' '.join([diagnosis_mapping.get(word, word).strip() for word in x.split()]))

for i, value in os_data['Diagnosis'].items():

    tokens = word_tokenize(value.lower())

    for j, token in enumerate(tokens):

        if token in diagnosis_mapping:

            tokens[j] = diagnosis_mapping[token]

    os_data.at[i, 'Diagnosis'] = ' '.join(tokens)

#print(os_data['Diagnosis'].value_counts(dropna = False))

gender_le = LabelEncoder()
diagnosis_le = LabelEncoder()
phakic_le = LabelEncoder()

#print("Number of od null values before imputation\n", od_data.isnull().sum())
#print("Number of os null values before imputation\n", os_data.isnull().sum())

od_data['Gender'] = gender_le.fit_transform(od_data['Gender'])
od_data['Diagnosis'] = diagnosis_le.fit_transform(od_data['Diagnosis'])
od_mask = od_data['Phakic/Pseudophakic'].isna()
od_data.loc[~od_mask, 'Phakic/Pseudophakic'] = phakic_le.fit_transform(od_data.loc[~od_mask, 'Phakic/Pseudophakic'])

os_data['Gender'] = gender_le.fit_transform(os_data['Gender'])
os_data['Diagnosis'] = diagnosis_le.fit_transform(os_data['Diagnosis'])
os_mask = os_data['Phakic/Pseudophakic'].isna()
os_data.loc[~os_mask, 'Phakic/Pseudophakic'] = phakic_le.fit_transform(os_data.loc[~os_mask, 'Phakic/Pseudophakic'])

imputer = IterativeImputer()
imputed_od = imputer.fit_transform(od_data)
imputed_os = imputer.fit_transform(os_data)

imputed_od = pd.DataFrame(imputed_od, columns=od_data.columns)
imputed_os = pd.DataFrame(imputed_os, columns=os_data.columns)

imputed_od['Gender'] = gender_le.inverse_transform(imputed_od['Gender'].astype(int))
imputed_od['Diagnosis'] = diagnosis_le.inverse_transform(imputed_od['Diagnosis'].astype(int))
imputed_od['Phakic/Pseudophakic'] = phakic_le.inverse_transform(imputed_od['Phakic/Pseudophakic'].astype(int))

imputed_os['Gender'] = gender_le.inverse_transform(imputed_os['Gender'].astype(int))
imputed_os['Diagnosis'] = diagnosis_le.inverse_transform(imputed_os['Diagnosis'].astype(int))
imputed_os['Phakic/Pseudophakic'] = phakic_le.inverse_transform(imputed_os['Phakic/Pseudophakic'].astype(int))

imputed_od.drop_duplicates()
imputed_os.drop_duplicates()

#print("Number of od null values after imputation\n", imputed_od.isnull().sum())
#print("Number of os null values after imputation\n", imputed_os.isnull().sum())

imputed_od.insert(0, 'ID', od_id)
imputed_os.insert(0, 'ID', os_id)

threshold = 3

#odbefore = imputed_od.shape
#osbefore = imputed_os.shape

for column in imputed_od.columns:

    if(imputed_od[column].dtype) == 'float64':

        od_mean = np.mean(imputed_od[column])
        os_mean = np.mean(imputed_os[column])

        od_std = np.std(imputed_od[column])
        os_std = np.std(imputed_os[column])

        od_z_scores = (imputed_od[column] - od_mean) / od_std
        os_z_scores = (imputed_os[column] - os_mean) / os_std

        od_outliers = np.abs(od_z_scores) > threshold
        os_outliers = np.abs(os_z_scores) > threshold

        imputed_od = imputed_od[~od_outliers]
        imputed_os = imputed_os[~os_outliers]

imputed_od['Phakic/Pseudophakic'] = imputed_od['Phakic/Pseudophakic'].replace({'the eye keeps the lens': 'lens kept', 'crystalline lens has been removed ': 'lens removed'})
imputed_os['Phakic/Pseudophakic'] = imputed_os['Phakic/Pseudophakic'].replace({'the eye keeps the lens': 'lens kept', 'crystalline lens has been removed ': 'lens removed'})

#print("od shape with outliers\n", odbefore)
#print("od shape outliers removed\n", imputed_od.shape)
#print("os shape with outliers\n", osbefore)
#print("os shape outliers removed\n", imputed_os.shape)

imputed_od.to_excel("od_cleaned.xlsx", index = False, header = False)
imputed_os.to_excel("os_cleaned.xlsx", index = False, header = False)

#load od_cleaned.xlsx and create a worksheet
od_cleaned_wb = load_workbook(filename = "od_cleaned.xlsx")
od_cleaned_ws = od_cleaned_wb["Sheet1"]

#load os_cleaned.xlsx and create a worksheet
os_cleaned_wb = load_workbook(filename = "os_cleaned.xlsx")
os_cleaned_ws = os_cleaned_wb["Sheet1"]

#insert a new row for the headers
od_cleaned_ws.insert_rows(1)
od_header_row = od_cleaned_ws[1]

os_cleaned_ws.insert_rows(1)
os_header_row = os_cleaned_ws[1]

#header titles
headers = ["ID", "Age", "Gender", "Diagnosis", "dioptre_1", "dioptre_2", "astigmatism", "Phakic/Pseudophakic", "Pneumatic", "Perkins", "Pachymetry", "Axial_Length", "VF_MD"]

#add the headers to the header row
for header in headers:

    od_header_row[headers.index(header)].value = header
    os_header_row[headers.index(header)].value = header

#loop through the cells in the header row
for cell in od_header_row:

    #format header row
    cell.font = openpyxl.styles.Font(bold = True)
    cell.alignment = openpyxl.styles.Alignment(horizontal = 'center')
    border = openpyxl.styles.Border(left=openpyxl.styles.Side(style='thin'),right=openpyxl.styles.Side(style='thin'),top=openpyxl.styles.Side(style='thin'),bottom=openpyxl.styles.Side(style='thin'))
    cell.border = border

#loop through the cells in the header row
for cell in os_header_row:

    #format header row
    cell.font = openpyxl.styles.Font(bold = True)
    cell.alignment = openpyxl.styles.Alignment(horizontal = 'center')
    border = openpyxl.styles.Border(left=openpyxl.styles.Side(style='thin'),right=openpyxl.styles.Side(style='thin'),top=openpyxl.styles.Side(style='thin'),bottom=openpyxl.styles.Side(style='thin'))
    cell.border = border

od_cleaned_ws.insert_rows(1)
od_grouped_row = od_cleaned_ws[1]

os_cleaned_ws.insert_rows(1)
os_grouped_row = od_cleaned_ws[1]

#grouped header titles
grouped_headers = ["Age", "Gender", "Diagnosis", "Refractive_Defect", "", "", "Phakic/Pseudophakic", "IOP", "", "Pachymetry", "Axial_Length", "VF_MD"]

#add the headers to the grouped headers row
for header in grouped_headers:

    od_grouped_row[grouped_headers.index(header) + 1].value = header
    os_grouped_row[grouped_headers.index(header) + 1].value = header

#loop through each cell in the grouped header row
for cell in od_grouped_row:

    #format the grouped headers
    cell.font = openpyxl.styles.Font(bold = True)
    cell.alignment = openpyxl.styles.Alignment(horizontal = 'center')
    border = openpyxl.styles.Border(left=openpyxl.styles.Side(style='thin'),right=openpyxl.styles.Side(style='thin'),top=openpyxl.styles.Side(style='thin'),bottom=openpyxl.styles.Side(style='thin'))
    cell.border = border

#loop through each cell in the grouped header row
for cell in os_grouped_row:

    #format the grouped headers
    cell.font = openpyxl.styles.Font(bold = True)
    cell.alignment = openpyxl.styles.Alignment(horizontal = 'center')
    border = openpyxl.styles.Border(left=openpyxl.styles.Side(style='thin'),right=openpyxl.styles.Side(style='thin'),top=openpyxl.styles.Side(style='thin'),bottom=openpyxl.styles.Side(style='thin'))
    cell.border = border

#load os sheet
oswb = load_workbook(filename = "os.xlsx")
osws = oswb["Sheet1"]

#merge cells to created grouped headers
od_cleaned_ws.merge_cells('E1:G1')
od_cleaned_ws.merge_cells('I1:J1')

#merge cells to created grouped headers
os_cleaned_ws.merge_cells('E1:G1')
os_cleaned_ws.merge_cells('I1:J1')

#loop through the columns in os.xslx
for column in osws.columns:

    #get the width of each column
    col_index = column[1].column_letter
    col_width = osws.column_dimensions[col_index].width

    #set column width for od equal to os columns
    if col_width == 13.0:

        od_cleaned_ws.column_dimensions[col_index].width = col_width /1.475
        os_cleaned_ws.column_dimensions[col_index].width = col_width /1.475

    else:
        #set column width for od equal to os columns
        od_cleaned_ws.column_dimensions[col_index].width = col_width
        os_cleaned_ws.column_dimensions[col_index].width = col_width

#save sheet to workbook
od_cleaned_wb.save('od_cleaned.xlsx')
os_cleaned_wb.save('os_cleaned.xlsx')

(3) Problem 3

In [ ]:
od = pd.read_excel("od_cleaned.xlsx")
od.drop(index=0, inplace=True)

os = pd.read_excel("os_cleaned.xlsx")
os.drop(index=0, inplace=True)

headers = ["ID", "Age", "Gender", "Diagnosis", "dioptre_1", "dioptre_2", "astigmatism", "Phakic/Pseudophakic", "Pneumatic", "Perkins", "Pachymetry", "Axial_Length", "VF_MD"]
numeric = ['Age', 'dioptre_1', 'dioptre_2', 'astigmatism', 'Pneumatic', 'Perkins', 'Pachymetry', 'Axial_Length', 'VF_MD']

od.columns = headers
os.columns = headers

od.drop('ID', axis = 1, inplace = True)
os.drop('ID', axis = 1, inplace = True)

for header in numeric:
    od[header] = pd.to_numeric(od[header])
    os[header] = pd.to_numeric(os[header])

od = od.drop(od[od['Diagnosis'] == 'suspicious'].index)
os = os.drop(os[os['Diagnosis'] == 'suspicious'].index)

plot1 = sns.catplot(data=od, x='Gender', hue='Diagnosis', kind = 'count', legend= False)
plot1.ax.legend(title = 'Diagnosis', fontsize=14, title_fontsize = 16)
plt.title("'od.xlsx': Gender vs Diagnosis",fontsize = 16, fontweight = 'bold')
plt.xlabel('Gender', fontdict={'fontsize': 14, 'fontweight': 'bold'})
plt.ylabel('count', fontdict={'fontsize': 14, 'fontweight': 'bold'})
plt.xticks(fontsize=14)

plot2 = sns.catplot(data=os, x='Gender', hue='Diagnosis', kind = 'count', legend = False)
plot2.ax.legend(title = 'Diagnosis', fontsize=14, title_fontsize = 16)
plt.title("'os.xlsx': Gender vs Diagnosis",fontsize = 16, fontweight = 'bold')
plt.xlabel('Gender', fontdict={'fontsize': 14, 'fontweight': 'bold'})
plt.ylabel('count', fontdict={'fontsize': 14, 'fontweight': 'bold'})
plt.xticks(fontsize=14)

plot3 = sns.catplot(data=od, x='Phakic/Pseudophakic', hue='Diagnosis', kind = 'count', order = ["lens kept", "lens removed"], legend = False)
plot3.ax.legend(title = 'Diagnosis', fontsize=14, title_fontsize = 16)
plt.title("'od.xlsx': Phakic/Pseudophakic vs Diagnosis",fontsize = 16, fontweight = 'bold')
plt.xlabel('Phakic/Pseudophakic', fontdict={'fontsize': 14, 'fontweight': 'bold'})
plt.ylabel('count', fontdict={'fontsize': 14, 'fontweight': 'bold'})
plt.xticks(fontsize=14)

plot4 = sns.catplot(data=os, x='Phakic/Pseudophakic', hue='Diagnosis', kind = 'count', order = ["lens kept", "lens removed"], legend = False)
plot4.ax.legend(title = 'Diagnosis', fontsize=14, title_fontsize = 16)
plt.title("'os.xlsx': Phakic/Pseudophakic vs Diagnosis",fontsize = 16, fontweight = 'bold')
plt.xlabel('Phakic/Pseudophakic', fontdict={'fontsize': 14, 'fontweight': 'bold'})
plt.ylabel('count', fontdict={'fontsize': 14, 'fontweight': 'bold'})
plt.xticks(fontsize=14)

In [ ]:
sns.pairplot(od, hue = 'Diagnosis', vars = numeric)

In [ ]:
sns.pairplot(os, hue = 'Diagnosis', vars = numeric)

In [ ]:
cat1 = sns.catplot(x='Diagnosis', y='Age', data=od, kind='violin')
plt.title("'od.xlsx': Age vs Diagnosis Distribution",fontsize = 14, fontweight = 'bold')
plt.xlabel('Diagnosis', fontdict={'fontsize': 14, 'fontweight': 'bold'})
plt.ylabel('Age', fontdict={'fontsize': 14, 'fontweight': 'bold'})
plt.xticks(fontsize=14)

cat2 = sns.catplot(x='Diagnosis', y='astigmatism', data=od, kind='violin')
plt.title("'od.xlsx': astigmatism vs Diagnosis Distribution",fontsize = 14, fontweight = 'bold')
plt.xlabel('Diagnosis', fontdict={'fontsize': 14, 'fontweight': 'bold'})
plt.ylabel('astigmatism', fontdict={'fontsize': 14, 'fontweight': 'bold'})
plt.xticks(fontsize=14)

cat3 = sns.catplot(x='Diagnosis', y='Pneumatic', data=od, kind='violin')
plt.title("'od.xlsx': Pneumatic vs Diagnosis Distribution",fontsize = 14, fontweight = 'bold')
plt.xlabel('Diagnosis', fontdict={'fontsize': 14, 'fontweight': 'bold'})
plt.ylabel('Pneumatic', fontdict={'fontsize': 14, 'fontweight': 'bold'})
plt.xticks(fontsize=14)

cat4 = sns.catplot(x='Diagnosis', y='Perkins', data=od , kind='violin')
plt.title("'od.xlsx': Perkins vs Diagnosis Distribution",fontsize = 14, fontweight = 'bold')
plt.xlabel('Diagnosis', fontdict={'fontsize': 14, 'fontweight': 'bold'})
plt.ylabel('Perkins', fontdict={'fontsize': 14, 'fontweight': 'bold'})
plt.xticks(fontsize=14)

cat5 = sns.catplot(x='Diagnosis', y='Axial_Length', data=od, kind='violin')
plt.title("'od.xlsx': Axial_Length vs Diagnosis Distribution",fontsize = 14, fontweight = 'bold')
plt.xlabel('Diagnosis', fontdict={'fontsize': 14, 'fontweight': 'bold'})
plt.ylabel('Axial_Length', fontdict={'fontsize': 14, 'fontweight': 'bold'})
plt.xticks(fontsize=14)

cat6 = sns.catplot(x='Diagnosis', y='VF_MD', data=od, kind='violin')
plt.title("'od.xlsx': VF_MD vs Diagnosis Distribution",fontsize = 14, fontweight = 'bold')
plt.xlabel('Diagnosis', fontdict={'fontsize': 14, 'fontweight': 'bold'})
plt.ylabel('VF_MD', fontdict={'fontsize': 14, 'fontweight': 'bold'})
plt.xticks(fontsize=14)

In [ ]:
cat7 = sns.catplot(x='Diagnosis', y='Age', data=os, kind='violin')
plt.title("'os.xlsx': Age vs Diagnosis Distribution",fontsize = 14, fontweight = 'bold')
plt.xlabel('Diagnosis', fontdict={'fontsize': 14, 'fontweight': 'bold'})
plt.ylabel('Age', fontdict={'fontsize': 14, 'fontweight': 'bold'})
plt.xticks(fontsize=14)

cat8 = sns.catplot(x='Diagnosis', y='dioptre_2', data=os, kind='violin')
plt.title("'os.xlsx': dioptre_2 vs Diagnosis Distribution",fontsize = 14, fontweight = 'bold')
plt.xlabel('Diagnosis', fontdict={'fontsize': 14, 'fontweight': 'bold'})
plt.ylabel('dioptre_2', fontdict={'fontsize': 14, 'fontweight': 'bold'})
plt.xticks(fontsize=14)

cat9 = sns.catplot(x='Diagnosis', y='Pneumatic', data=os, kind='violin')
plt.title("'os.xlsx': Pneumatic vs Diagnosis Distribution",fontsize = 14, fontweight = 'bold')
plt.xlabel('Diagnosis', fontdict={'fontsize': 14, 'fontweight': 'bold'})
plt.ylabel('Pneumatic', fontdict={'fontsize': 14, 'fontweight': 'bold'})
plt.xticks(fontsize=14)

cat10 = sns.catplot(x='Diagnosis', y='VF_MD', data=os, kind='violin')
plt.title("'os.xlsx': VF_MD vs Diagnosis Distribution",fontsize = 14, fontweight = 'bold')
plt.xlabel('Diagnosis', fontdict={'fontsize': 14, 'fontweight': 'bold'})
plt.ylabel('VF_MD', fontdict={'fontsize': 14, 'fontweight': 'bold'})
plt.xticks(fontsize=14)

In [ ]:
scatter1 = sns.scatterplot(data=od, x='Perkins', y='Pachymetry', hue = 'Diagnosis')
plt.title("'od.xlsx': Perkins vs Pachymetry Scatter",fontsize = 14, fontweight = 'bold')
plt.xlabel('Perkins', fontdict={'fontsize': 14, 'fontweight': 'bold'})
plt.ylabel('Pachymetry', fontdict={'fontsize': 14, 'fontweight': 'bold'})
plt.legend(title = "Diagnosis" ,loc='lower right')
plt.xticks(fontsize=14)

In [ ]:
scatter2 = sns.scatterplot(data=os, x='Perkins', y='Pachymetry', hue = 'Diagnosis')
plt.title("'os.xlsx': Perkins vs Pachymetry Scatter",fontsize = 14, fontweight = 'bold')
plt.xlabel('Perkins', fontdict={'fontsize': 14, 'fontweight': 'bold'})
plt.ylabel('Pachymetry', fontdict={'fontsize': 14, 'fontweight': 'bold'})
plt.legend(title = "Diagnosis", loc='lower right')
plt.xticks(fontsize=14)


In [ ]:
box = sns.catplot(x='Diagnosis', y='Axial_Length', data=od, kind='box')
plt.title("'od.xlsx': Axial_Length vs Diagnosis Box",fontsize = 14, fontweight = 'bold')
plt.xlabel('Diagnosis', fontdict={'fontsize': 14, 'fontweight': 'bold'})
plt.ylabel('Axial_Length', fontdict={'fontsize': 14, 'fontweight': 'bold'})
plt.xticks(fontsize=14)

box2 = sns.catplot(x='Diagnosis', y='Axial_Length', data=os, kind='box')
plt.title("'os.xlsx': Axial_Length vs Diagnosis Box",fontsize = 14, fontweight = 'bold')
plt.xlabel('Diagnosis', fontdict={'fontsize': 14, 'fontweight': 'bold'})
plt.ylabel('Axial_Length', fontdict={'fontsize': 14, 'fontweight': 'bold'})
plt.xticks(fontsize=14)